In [6]:
import torch
from PIL import Image
import torchvision.transforms as T
import os

In [2]:
model = torch.load("./skin-model-pokemon.pt", map_location=torch.device('cpu'))

In [4]:
model.eval()

EfficientNet(
  (features): Sequential(
    (0): ConvNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): ConvNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): ConvNormActivation(
 

In [7]:
data_dir = './datasets'
classes = os.listdir(data_dir)
classes

['acanthosis-nigricans',
 'acne',
 'acne-scars',
 'alopecia-areata',
 'dry',
 'melasma',
 'oily',
 'vitiligo',
 'warts']

In [10]:
# Test on an image
img = Image.open("./datasets/warts/warts(14).jpg").convert("RGB")

In [11]:
def get_transforms():
    transform = []
    transform.append(T.Resize((512, 512)))
    transform.append(T.ToTensor())
    return T.Compose(transform)

In [12]:
tr = get_transforms()

In [25]:
img_tensor = tr(img)

In [20]:
img_tensor

tensor([[[0.2706, 0.2941, 0.2745,  ..., 0.4039, 0.3961, 0.3922],
         [0.2353, 0.2863, 0.2902,  ..., 0.4078, 0.4000, 0.4000],
         [0.1882, 0.2667, 0.2863,  ..., 0.4353, 0.4157, 0.4118],
         ...,
         [0.8039, 0.8078, 0.8118,  ..., 0.6157, 0.6392, 0.6902],
         [0.8039, 0.8000, 0.8039,  ..., 0.5529, 0.5804, 0.6235],
         [0.8000, 0.8039, 0.8039,  ..., 0.5608, 0.5843, 0.5765]],

        [[0.2392, 0.2471, 0.2275,  ..., 0.3608, 0.3569, 0.3569],
         [0.2078, 0.2431, 0.2431,  ..., 0.3647, 0.3569, 0.3569],
         [0.1686, 0.2353, 0.2431,  ..., 0.3922, 0.3686, 0.3686],
         ...,
         [0.7765, 0.7804, 0.7843,  ..., 0.5647, 0.5843, 0.6392],
         [0.7725, 0.7725, 0.7765,  ..., 0.5020, 0.5294, 0.5725],
         [0.7725, 0.7725, 0.7765,  ..., 0.5137, 0.5373, 0.5333]],

        [[0.1647, 0.1804, 0.1686,  ..., 0.2980, 0.2980, 0.2980],
         [0.1490, 0.1804, 0.1804,  ..., 0.2980, 0.2941, 0.2941],
         [0.1216, 0.1725, 0.1843,  ..., 0.3255, 0.3059, 0.

In [21]:
img_tensor.shape

torch.Size([3, 512, 512])

In [24]:
out = model(img_tensor.unsqueeze(0))

In [26]:
out

tensor([[-1.8207, -5.2925, -0.1130, -1.5689,  1.2904, -1.3440, -3.1785,  2.2026,
          9.6765]], grad_fn=<AddmmBackward0>)

In [33]:
pred, idx = torch.max(out, 1)

In [34]:
pred

tensor([9.6765], grad_fn=<MaxBackward0>)

In [35]:
classes[idx]

'warts'

In [37]:
def predict(model, img, tr, classes):
    img_tensor = tr(img)
    out = model(img_tensor.unsqueeze(0))
    pred, idx = torch.max(out, 1)
    return classes[idx]

In [38]:
img = Image.open('./datasets/dry/dry(48).jpg').convert("RGB")
res = predict(model, img, tr, classes)
res

'dry'

In [39]:
img = Image.open('./datasets/acne-scars/acne-scars(7)(1).jpg').convert("RGB")
res = predict(model, img, tr, classes)
res

'acne-scars'